# Google Drive をマウントする

In [1]:
# 必要なライブラリを読み込む
from google.colab import drive 

# Google ドライブをマウントする
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# 必要なライブラリのインポート

In [2]:
# 必要なライブラリを読み込む

# データフレームが扱えるよう、pandas をインポート
import pandas as pd

# numpy のデータが扱えるよう、numpy をインポート
import numpy as np

# データ分割を行う便利なモジュール train_test_split をインポート
from sklearn.model_selection import train_test_split

# グリッドサーチを行う便利なモジュール train_test_split をインポート
from sklearn.model_selection import GridSearchCV

# アンダーサンプリング用ライブラリをインポート
from imblearn.under_sampling import RandomUnderSampler

# データの標準化を行うためのモジュールをインポート
from sklearn.preprocessing import StandardScaler

# 機械学習用ライブラリ sklearn（scikit-learn）から線形回帰用クラス linear_model 中の
# Lasso回帰用クラス Lasso をインポート 
from sklearn.linear_model import Lasso

# 統計解析用ライブラリ statsmodels 内にあるライブラリ stats.outliers_influence から
# 分散拡大係数（VIF）計算用メソッド variance_inflation_factor をインポート
from statsmodels.stats.outliers_influence import variance_inflation_factor

# ヒストグラムを描くためのモジュールをインポート
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 準備

# データ読み込み

あるカフェの売上と、そのカフェの店舗属性（最寄駅からの距離、競合店舗の数など）について収録したデータになります（CAFE.csv）。
ここでは、店舗属性をつかって売上額（SALES）を予測するという課題に取り組んでいただきます。
*   説明変数（店舗属性）
> *   数値変数は STATION_USER、MINUTES_FROM_STATION、COMPETITOR、AREA、SHEETS、LEADER_AGE、WORKERS、MENU、ADS の全 9 種類。
*   目的変数（売上額）
> *   SALES

In [3]:
# カフェ店舗属性・売上データを読み込む
cafe = pd.read_csv("drive/My Drive/DATA-SCI-TECH-EXERCISE/DATA/CAFE.csv")
cafe.describe()

,STATION_USER,MINUTES_FROM_STATION,COMPETITOR,AREA,SHEETS,LEADER_AGE,WORKERS,MENU,ADS,SALES
count,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,1.280000e+02
mean,78191.609375,1.148438,3.062500,54.695312,28.695312,30.070312,19.234375,20.500000,302335.148438,4.353853e+06
std,21520.811263,0.870242,0.858033,10.859561,5.642302,2.074151,4.024501,3.324924,50269.011049,7.721717e+05
min,31404.000000,0.000000,2.000000,29.000000,16.000000,26.000000,11.000000,14.000000,171116.000000,2.686770e+06
25%,62155.250000,1.000000,2.000000,47.000000,25.000000,29.000000,16.000000,18.000000,271002.750000,3.838247e+06
50%,78999.500000,1.000000,3.000000,54.000000,29.000000,30.000000,19.000000,20.000000,298047.500000,4.360999e+06
75%,94851.000000,2.000000,4.000000,61.000000,32.250000,31.000000,22.000000,23.000000,330986.000000,4.786499e+06
max,136056.000000,4.000000,5.000000,86.000000,47.000000,36.000000,27.000000,27.000000,418527.000000,6.793403e+06


In [4]:
# 概要を確認
cafe.describe()

,STATION_USER,MINUTES_FROM_STATION,COMPETITOR,AREA,SHEETS,LEADER_AGE,WORKERS,MENU,ADS,SALES
count,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,1.280000e+02
mean,78191.609375,1.148438,3.062500,54.695312,28.695312,30.070312,19.234375,20.500000,302335.148438,4.353853e+06
std,21520.811263,0.870242,0.858033,10.859561,5.642302,2.074151,4.024501,3.324924,50269.011049,7.721717e+05
min,31404.000000,0.000000,2.000000,29.000000,16.000000,26.000000,11.000000,14.000000,171116.000000,2.686770e+06
25%,62155.250000,1.000000,2.000000,47.000000,25.000000,29.000000,16.000000,18.000000,271002.750000,3.838247e+06
50%,78999.500000,1.000000,3.000000,54.000000,29.000000,30.000000,19.000000,20.000000,298047.500000,4.360999e+06
75%,94851.000000,2.000000,4.000000,61.000000,32.250000,31.000000,22.000000,23.000000,330986.000000,4.786499e+06
max,136056.000000,4.000000,5.000000,86.000000,47.000000,36.000000,27.000000,27.000000,418527.000000,6.793403e+06


# データ準備

In [5]:
# 説明変数と目的変数に分離する
columnList = ["STATION_USER","MINUTES_FROM_STATION","COMPETITOR","AREA","SHEETS","LEADER_AGE","WORKERS","MENU","ADS"]
X = pd.DataFrame( cafe.loc[:, columnList] )
y = pd.DataFrame( cafe.loc[:, ["SALES"]] )

# 標準化の実施

In [6]:
# X_train のデータを使い標準化パラメータを獲得してから、X_train、X_test に対して標準化を実施
scaler_X = StandardScaler()
scaler_X.fit( X )
X_std = pd.DataFrame(scaler_X.transform(X), columns=columnList)

In [7]:
# y_train のデータを使い標準化パラメータを獲得してから、y_train、y_test に対して標準化を実施
scaler_y = StandardScaler()
scaler_y.fit( y )
y_std = pd.DataFrame(scaler_y.transform(y), columns=["accuracy"])

# モデル構築（Lasso回帰）

クロスバリデーションを実施しながら、Lasso回帰モデルをつくります。
以下の手順に従います。
1. パラメータを入力します。
2. 評価関数を入力します
3. 解析を実施します。

In [8]:
# サーチする内容を定義する
search_params = [ { 'alpha':[0.1], 'random_state' : [1234] } ]

In [9]:
# 評価関数を定義する
score = 'r2'

In [10]:
# 分析条件をセットします
reg = GridSearchCV(
    Lasso(),        # 識別器
    search_params,  # 最適化したいパラメータセット 
    cv=5,           # 交差検定の回数
    scoring=score   # モデルの評価関数の指定
)

In [11]:
# クロスバリデーションによる分析を実施します
reg.fit(X_std,y_std.values.ravel())

GridSearchCV(cv=5, error_score=nan,
             estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=1000, normalize=False, positive=False,
                             precompute=False, random_state=None,
                             selection='cyclic', tol=0.0001, warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'alpha': [0.1], 'random_state': [1234]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='r2', verbose=0)

In [12]:
# 結果の一覧をみる
pd.DataFrame(reg.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_random_state,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003808,0.002334,0.0013,0.000437,0.1,1234,"{'alpha': 0.1, 'random_state': 1234}",0.83818,0.155079,0.764836,0.84284,0.832233,0.686633,0.267294,1


# 結果確認

構築されたモデルを確認します。

In [13]:
# 投入したデータセットの全ての変数についてVIFを計算する関数 checkVIF の定義
def checkVIF( ExplanatoryVarDataSet ):
  tmp_columnList = ExplanatoryVarDataSet.columns
  vifList = []
  for i in range(len(tmp_columnList)):
    colname = tmp_columnList[i]
    vif = variance_inflation_factor(ExplanatoryVarDataSet.values, i)
    vifList.append( [ colname, vif ] )
  return  pd.DataFrame( vifList, columns=["COLUMN","VIF"] )

In [14]:
# 偏回帰係数とともに分散拡大係数により多重共線性を確認する
vif = checkVIF(X_std)
vif["Lasso"] = reg.best_estimator_.coef_

# 切片の行を加えておく
intercepts = pd.DataFrame( [["intercept", 0, reg.best_estimator_.intercept_ ]] )
intercepts.columns = ["COLUMN","VIF","Lasso"]
vif = vif.append(intercepts)

# 表示する
vif

,COLUMN,VIF,Lasso
0,STATION_USER,1.053841,5.012904e-01
1,MINUTES_FROM_STATION,1.050553,2.241842e-01
2,COMPETITOR,1.117270,-5.173470e-02
3,AREA,42.255662,3.991778e-01
4,SHEETS,41.832691,0.000000e+00
5,LEADER_AGE,1.055814,-0.000000e+00
6,WORKERS,1.047041,0.000000e+00
7,MENU,1.096382,0.000000e+00
8,ADS,1.034284,0.000000e+00
0,intercept,0.000000,6.209229e-18
